# imports et detection des IDs + ouverture instruments

In [ ]:
import pyvisa
import time

rm = pyvisa.ResourceManager()
resources = rm.list_resources()
print("Ressources trouvées :", resources)

generator_addr = None
scope_addr = None

for addr in resources:
    # Générateur Rigol (souvent DGxxxx / DG1D)
    if "DG1D" in addr or "DG" in addr:
        generator_addr = addr
    # Oscillo Rigol (souvent DSxxxx / DS1Z)
    elif "DS1Z" in addr or "DS" in addr:
        scope_addr = addr

print("Générateur :", generator_addr)
print("Oscilloscope :", scope_addr)

# Ouvrir les instruments (si trouvés)
if generator_addr is None:
    raise RuntimeError("Générateur non trouvé dans rm.list_resources()")

generator = rm.open_resource(generator_addr)
generator.timeout = 10000
generator.read_termination = '\n'
generator.write_termination = '\n'
generator.chunk_size = 102400
generator.clear()

# L'oscillo peut être optionnel si tu n'en as pas besoin tout de suite
scope = None
if scope_addr is not None:
    scope = rm.open_resource(scope_addr)
    scope.timeout = 10000
    scope.read_termination = '\n'
    scope.write_termination = '\n'
    scope.chunk_size = 102400
    scope.clear()

print("OK: instruments ouverts.")


# Fonctions utilitaires

In [ ]:
def gen_write(cmd: str, pause: float = 0.05):
    """Envoie une commande au générateur avec petite pause (évite les soucis de buffer)."""
    generator.write(cmd)
    time.sleep(pause)

def ch_on(ch: int):
    """Active la sortie du canal ch (1 ou 2)."""
    if ch == 1:
        gen_write("OUTP1 ON")
    elif ch == 2:
        gen_write("OUTP2 ON")
    else:
        raise ValueError("Canal invalide (utilise 1 ou 2)")
    print(f"CH{ch} : ON")

def ch_off(ch: int):
    """Désactive la sortie du canal ch (1 ou 2)."""
    if ch == 1:
        gen_write("OUTP1 OFF")
    elif ch == 2:
        gen_write("OUTP2 OFF")
    else:
        raise ValueError("Canal invalide (utilise 1 ou 2)")
    print(f"CH{ch} : OFF")

def all_off():
    """Coupe CH1 et CH2 rapidement."""
    ch_off(1)
    ch_off(2)


# Test prise en main 

In [ ]:
# Test CH1
ch_on(1)
time.sleep(1)
ch_off(1)
time.sleep(0.5)

# Test CH2
ch_on(2)
time.sleep(1)
ch_off(2)
time.sleep(0.5)

print("Test ON/OFF terminé.")


# Envoi du signal (SIN ou RAMP sur CH1, et SQUARE sur CH2)

In [ ]:
# ====== CH1 : choisir une forme d'onde ======

# --- Option 1 : SINUS (fréquence Hz, Vpp, offset) ---
gen_write("APPL:SIN 1000,5.0,-1.5")  # 1 kHz, 5 Vpp, offset -1.5 V
# ch_on(1)

# --- Option 2 : RAMPE (RAMP) ---
# gen_write("APPL:RAMP 1000,5.0,-1.5")  # 1 kHz, 5 Vpp, offset -1.5 V
# ch_on(1)

# Active CH1 (garde une seule ligne ch_on(1) en vrai)
ch_on(1)
print("CH1 -> signal configuré et activé.")


# ====== CH2 : SQUARE ======
gen_write("APPLy:SQUare:CH2 1000,3.0,0.0")  # 1 kHz, 3 Vpp, offset 0 V
ch_on(2)
print("CH2 -> SQUARE configuré et activé.")


# stop 

In [ ]:
all_off()
print("Sorties coupées.")


« Ce code me permet de prendre le contrôle du générateur de fonctions via VISA.
Je peux activer ou désactiver chaque canal et envoyer des signaux sinusoïdaux, rampes ou carrés avec des paramètres précis (fréquence, amplitude, offset).
Cette étape sert uniquement à configurer la source du signal avant toute mesure à l’oscilloscope. »

# OSCILLO : lecture CH1/CH2 + mesures + plot

In [ ]:
import pyvisa
import numpy as np
import matplotlib.pyplot as plt

SCOPE_ID = "USB0::0x1AB1::0x04CE::DS1ZC212301111::INSTR"


# Helpers: parsing binaire Rigol (#<n><len><data>)
def _parse_ieee_block(raw: bytes) -> bytes:
    """
    Parse un bloc IEEE488.2 binaire: b'#' + n + <len> + <data>
    Retourne uniquement <data>.
    """
    if not raw or raw[0:1] != b'#':
        # parfois certains scopes renvoient ASCII si config différente
        return raw

    n_digits = int(raw[1:2].decode(errors="ignore"))
    if n_digits <= 0:
        return raw

    len_start = 2
    len_end = 2 + n_digits
    data_len = int(raw[len_start:len_end].decode(errors="ignore"))
    data_start = len_end
    data_end = data_start + data_len
    return raw[data_start:data_end]


# Lecture waveform + scaling via PREamble
def read_waveform(scope, channel="CHAN1", max_points=None):
    """
    Lit une waveform sur un Rigol DS1Z/DS1000Z et renvoie (t, v).
    - Ne fait PAS de :STOP
    - Ne fait PAS de *IDN?
    - Configure le minimum pour lire la waveform
    """
    # Source
    scope.write(f":WAV:SOUR {channel}")

    # Format binaire le plus robuste (1 byte/point)
    # (oui, c'est une "config", mais c'est la plus safe pour lire des données)
    scope.write(":WAV:FORM BYTE")

    # Mode NORM = points affichés (plus stable que RAW pour début)
    scope.write(":WAV:MODE NORM")

    # Optionnel: limiter nb points si tu veux aller vite
    if max_points is not None:
        # Certains Rigol acceptent :WAV:POIN <n>
        # Si ça échoue, ça n'empêche pas la lecture
        try:
            scope.write(f":WAV:POIN {int(max_points)}")
        except Exception:
            pass

    # PREamble: scaling
    # format: FORMAT,TYPE,POINTS,COUNT,XINCR,XORIG,XREF,YINCR,YORIG,YREF
    pre = scope.query(":WAV:PRE?").strip()
    parts = pre.split(',')
    if len(parts) < 10:
        raise RuntimeError(f"PREamble inattendue: {pre}")

    points = int(float(parts[2]))
    xincr  = float(parts[4])
    xorig  = float(parts[5])
    xref   = float(parts[6])
    yincr  = float(parts[7])
    yorig  = float(parts[8])
    yref   = float(parts[9])

    # DATA
    scope.write(":WAV:DATA?")
    raw = scope.read_raw()   # bytes
    data = _parse_ieee_block(raw)

    # Convert bytes -> volts
    # v = (byte - yref) * yincr + yorig
    y = np.frombuffer(data, dtype=np.uint8).astype(np.float64)
    v = (y - yref) * yincr + yorig

    # Temps
    # t = (i - xref)*xincr + xorig
    i = np.arange(v.size, dtype=np.float64)
    t = (i - xref) * xincr + xorig

    # (Parfois v.size != points, on s'aligne sur ce qui est reçu)
    return t, v


# Mesures numériques
def compute_characteristics(t, v):
    """
    Calcule quelques caractéristiques simples.
    """
    vmin = float(np.min(v))
    vmax = float(np.max(v))
    vpp  = vmax - vmin
    vmean = float(np.mean(v))
    vrms  = float(np.sqrt(np.mean(v**2)))

    # Estimation fréquence par passages à zéro autour de la moyenne
    vc = v - vmean
    sign = np.sign(vc)
    zc = np.where(np.diff(sign) != 0)[0]  # indices où ça traverse
    freq = float("nan")
    if len(zc) >= 3:
        # On prend des intervalles entre traversées similaires (2 passages = 1 période approx)
        tz = t[zc]
        # période approx = moyenne de (tz[k+2]-tz[k])
        periods = tz[2:] - tz[:-2]
        T = np.mean(periods) if len(periods) > 0 else np.nan
        if T and T > 0:
            freq = float(1.0 / T)

    return {
        "Vmin": vmin,
        "Vmax": vmax,
        "Vpp": float(vpp),
        "Vmean": vmean,
        "Vrms": vrms,
        "Freq_est": freq
    }

def detect_saturation(v, flat_ratio_threshold=0.12):
    """
    Détection simple de saturation: présence de "plateaux" proches de Vmax/Vmin.
    - On calcule la proportion d'échantillons très proches des extrêmes.
    """
    vmin = np.min(v); vmax = np.max(v)
    vpp = vmax - vmin
    if vpp <= 1e-12:
        return True

    eps = 0.02 * vpp  # 2% du Vpp comme bande près des rails
    near_top = np.mean(v >= (vmax - eps))
    near_bot = np.mean(v <= (vmin + eps))

    # Si une grosse fraction est collée près d'un extrême -> plateau -> saturation probable
    return (near_top > flat_ratio_threshold) or (near_bot > flat_ratio_threshold)


# MAIN
rm = pyvisa.ResourceManager()
scope = rm.open_resource(SCOPE_ID)

scope.timeout = 10000
scope.read_termination = '\n'
scope.write_termination = '\n'
scope.chunk_size = 102400
scope.clear()

# Lire CH1 et CH2
t1, v1 = read_waveform(scope, "CHAN1")
t2, v2 = read_waveform(scope, "CHAN2")
t = t1  # base de temps

# Caractéristiques
car1 = compute_characteristics(t1, v1)
car2 = compute_characteristics(t2, v2)

print("\n=== CH1 (entrée) ===")
for k, val in car1.items():
    print(f"{k} = {val}")

print("\n=== CH2 (sortie) ===")
for k, val in car2.items():
    print(f"{k} = {val}")

# Gain LM741 (sur Vpp)
if car1["Vpp"] > 0:
    gain_lin = car2["Vpp"] / car1["Vpp"]
    gain_db  = 20 * np.log10(abs(gain_lin))
else:
    gain_lin = float("nan")
    gain_db  = float("nan")

print(f"\nGain (Vpp_out / Vpp_in) = {gain_lin:.3f}  ({gain_db:.2f} dB)")

# Saturation
sat1 = detect_saturation(v1)
sat2 = detect_saturation(v2)
print(f"Saturation CH1 ? {sat1}")
print(f"Saturation CH2 ? {sat2}")

# Plot
plt.figure(figsize=(10,5))
plt.plot(t*1e3, v1, label="CH1 (entrée)")
plt.plot(t*1e3, v2, label="CH2 (sortie)")
plt.xlabel("Temps (ms)")
plt.ylabel("Tension (V)")
plt.title("LM741 – entrée / sortie")
plt.grid(True)
plt.legend()
plt.show()

scope.close()
rm.close()
